In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import sin, cos, exp
from numpy.linalg import inv
np.random.seed(8008)

from scipy.integrate import odeint

In [2]:
#Fix parameters and initial conditions
N = 4000
dt = 0.02
iterate = 10
t0 = 0
t1 = N*dt
init = [5,5,5]

In [3]:
#Lorenz system
def get_lorenz_vals(dt, ic):
    def lorenz1(xyz, t, s=10, r=28, b=8/3):
        x,y,z = xyz
        dx = s*(y - x)
        dy = r*x - y - x*z
        dz = x*y - b*z
        return np.array([dx,dy,dz])
    
    from scipy.integrate import odeint
    sol = odeint(lorenz1, ic, dt)
    xs,ys,zs = sol.T

    return sol.T

In [4]:
#Implement the Algorithm
def find_approximation(system: callable, t0: float, t1: float,
                       N=N, D_r=100, w=0.005, b=4, beta=1e-5): 
    ic = init #initial condition = [5,5,5]
    U: np.ndarray = system(np.linspace(t0, t1, N+1), ic)
    D = U.shape[0]
    U_o = U[:, 1:]
    U_i = U[:, :-1]
    W_in = np.random.uniform(-w, w, (D_r, D))
    b_in = np.random.uniform(-b, b, (D_r,1))
    Phi = np.tanh(W_in @ U_i + b_in)
    W_LR = (U_o @ Phi.T @ inv(Phi @ Phi.T + beta * np.identity(D_r)))

    U_hat = np.atleast_2d(U[:, 0]).T
    for _ in range(N):
        u_n = U_hat[:, -1]
        phi = np.tanh(np.atleast_2d(W_in @ u_n).T + b_in)
        u_np1 = W_LR @ phi
        U_hat = np.concatenate((U_hat, u_np1), axis=1)
        
    U_pred = np.atleast_2d(U_hat[:, -1]).T # Further approximation starting from u_(n+1), will not be used in this experiment
    #for _ in range(N):
        #u_n = U_pred[:, -1]
        #phi = np.tanh(np.atleast_2d(W_in @ u_n).T + b_in)
        #u_np1 = W_LR @ phi
        #U_pred = np.concatenate((U_pred, u_np1), axis=1)
    #return U, U_hat, U_pred
    return U, U_hat, U_pred

In [5]:
def find_tf(system, N, D_r, β=1e-5, t0 = t0, t1 = t1):
    U, U_hat, U_pred = find_approximation(system, t0, t1, N=N, D_r=D_r, beta = β)
    error = 0
    tf = 0
    n=0
    while error <= 0.05:
        error = (np.linalg.norm(U[:,n]-U_hat[:,n]) / np.linalg.norm(U[:,n]))**2
        tf += dt
        n += 1
    return tf

find_tf(get_lorenz_vals, N, 1000, β=0.0000000001)

C:\Users\User\AppData\Local\Temp\ipykernel_18204\395627617.py:12: RuntimeWarning: overflow encountered in matmul
  W_LR = (U_o @ Phi.T @ inv(Phi @ Phi.T + beta * np.identity(D_r)))
C:\Users\User\AppData\Local\Temp\ipykernel_18204\395627617.py:12: RuntimeWarning: invalid value encountered in matmul
  W_LR = (U_o @ Phi.T @ inv(Phi @ Phi.T + beta * np.identity(D_r)))


0.04

In [ ]:
logdr = np.arange(2,8,1)
dr = 10**(logdr)

tf = np.zeros(len(dr))
for i in range(iterate):
    tf0 = []
    for d in dr:
        tf0.append(find_tf(get_lorenz_vals, N, d))
    tf = tf + np.array(tf0)
tf = tf/iterate

plt.plot(logdr, tf)

C:\Users\User\AppData\Local\Temp\ipykernel_18204\395627617.py:12: RuntimeWarning: overflow encountered in matmul
  W_LR = (U_o @ Phi.T @ inv(Phi @ Phi.T + beta * np.identity(D_r)))
C:\Users\User\AppData\Local\Temp\ipykernel_18204\395627617.py:12: RuntimeWarning: invalid value encountered in matmul
  W_LR = (U_o @ Phi.T @ inv(Phi @ Phi.T + beta * np.identity(D_r)))


In [ ]:
dr